# Edge detection: filtre de Sobel, filtre de Canny et la transformée de Hough

## Filtre de Sobel
[*vidéo intéressante de computerphile*](https://www.youtube.com/watch?v=uihBwtPIBxM)\
La détection des bords par la méthode de Sobel s'appuie sur 2 filtres:
$$
\mathbf{G_x}
=
\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}
$$
pour la détection des bords suivant la direction$x$, et

$$
\mathbf{G_y}
=
\begin{bmatrix}
-1 & -2 & -1 \\
 0 &  0 &  0 \\
 1 &  2 &  1
\end{bmatrix}
$$
pour la direction $y$.

Prenons une image d'une grille de sudoku:\
![sudoku](sudoku_grid.png)

In [ ]:
import numpy as np
import cv2
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
img = Image.open('sudoku_grid.png')
imgGray = np.array(img.convert('L'), dtype='float')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.imshow(imgGray, cmap="gray")
plt.show()

In [ ]:
# Définition des kernel Gx et Gy:
Gx = np.array([[-1, 0, 1],
               [-2, 0, 2],
               [-1, 0, 1]])
Gy = np.array([[-1, -2, -1],
               [ 0,  0,  0],
               [ 1,  2,  1]])

In [ ]:
# Observons dans un premier temps l'influence de l'application d'un flou Gaussien avant l'application du filtre de Sobel:
blur_img = cv2.GaussianBlur(imgGray, ksize=(5,5), sigmaX=3.0)


In [ ]:
# Application du filtre Gx
img_Gx = cv2.filter2D(imgGray, ddepth=-1, kernel=Gx)
blured_gauss_Gx = cv2.filter2D(blur_img, ddepth=-1, kernel=Gx)

# Application du filtre Gy
img_Gy = cv2.filter2D(imgGray, ddepth=-1, kernel=Gy)
blured_gauss_Gy = cv2.filter2D(blur_img, ddepth=-1, kernel=Gy)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(30,20))
axes[0][0].imshow(imgGray, cmap="gray")
axes[0][1].imshow(img_Gx, cmap="gray")
axes[0][2].imshow(blured_gauss_Gx, cmap="gray")

axes[1][0].imshow(imgGray, cmap="gray")
axes[1][1].imshow(img_Gy, cmap="gray")
axes[1][2].imshow(blured_gauss_Gy, cmap="gray")

axes[0][0].set_title("Gray")
axes[0][1].set_title("Gray * Gx")
axes[0][2].set_title("Gray * Gaussian * Gx")
axes[1][0].set_title("Gray")
axes[1][1].set_title("Gray * Gy")
axes[1][2].set_title("Gray * Gaussian * Gy")

plt.show()

In [ ]:
# Amplitude du vecteur obtenue par le filtre de Sobel et celui ayant subi un floutage Gaussien et le filtre de Sobel
amplitude_sobel = np.sqrt(img_Gx**2 + img_Gy**2)
amplitude_gauss_sobel = np.sqrt(blured_gauss_Gx**2 + blured_gauss_Gy**2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(22,6))
g0 = axes[0].imshow(amplitude_sobel, cmap="coolwarm", vmin=0, vmax=500)
g1 = axes[1].imshow(amplitude_gauss_sobel, cmap="coolwarm", vmin=0, vmax=500)
g2 = axes[2].imshow(amplitude_sobel - amplitude_gauss_sobel, cmap="gray", vmin=0, vmax=500)

axes[0].set_title("Amplitude gray convolved with Sobel")
axes[1].set_title("Amplitude gray blured + convolved with Sobel")
axes[2].set_title("Difference")

fig.colorbar(g0, ax=axes[0])
fig.colorbar(g1, ax=axes[1])
fig.colorbar(g2, ax=axes[2])

plt.show()

Nous pouvons grossièrement voir que les bords sont plus fort dans le cas où le flou Gaussien n'est pas appliqué.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,6))
g0 = axes[0].imshow(amplitude_sobel[25:150,150:250], cmap="coolwarm", vmin=0, vmax=500)
g1 = axes[1].imshow(amplitude_gauss_sobel[25:150,150:250], cmap="coolwarm", vmin=0, vmax=500)
g2 = axes[2].imshow((amplitude_sobel - amplitude_gauss_sobel)[25:150,150:250], cmap="gray", vmin=0, vmax=500)

axes[0].set_title("Amplitude gray convolved with Sobel")
axes[1].set_title("Amplitude gray blured + convolved with Sobel")
axes[2].set_title("Difference")

plt.show()

Si nous zoomons, nous pouvons voir plusieurs choses:
* Les zones correspondant au fond, par exemple les zones représentant un case de sudoku vierge, ont moins de bruit sur l'image ayant subi un flou gaussien que l'image ne l'ayant pas subi.
* Les bords des petits éléments comme les écritures au dessus de la grille sont plus estompés. Probablement dû à l'application du filtre Gaussien. Pour le confirmer nous pouvons comparer la netteté des écritures en fonction de la taille du kernel.
* Les bords de la grille sont plus diffus dans le cas de l'application du flou gaussien (observation déjà mentionnée)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,8))
tmp_arr = False
for i, size in enumerate([3, 5, 7, 9]):
    blur_img = cv2.GaussianBlur(imgGray, ksize=(size,size), sigmaX=2.0)
    blured_gauss_Gx = cv2.filter2D(blur_img, ddepth=-1, kernel=Gx)
    blured_gauss_Gy = cv2.filter2D(blur_img, ddepth=-1, kernel=Gy)
    amplitude_gauss_sobel = np.sqrt(blured_gauss_Gx**2 + blured_gauss_Gy**2)

    g = axes[i].imshow(amplitude_gauss_sobel[45:75,170:250], cmap="coolwarm")
    axes[i].set_title(f"kernel size of Gaussian blur, k={size}")

plt.show()

L'influence de la taille du kernel n'est pas très flagrante, on constate sur les bords horizontal un élargissement et un flou plus prononcé. Cela est tout de même concordant avec le fait que les petites écritures au dessus de la grille sont plus affaiblies/diluées avec la taille du kernel.

In [ ]:
# Observons dans un premier temps l'influence de l'application d'un flou Gaussien avant l'application du filtre de Sobel:
blur_img = cv2.GaussianBlur(imgGray, ksize=(9,9), sigmaX=3.0)

# Application du filtre Gx
img_Gx = cv2.filter2D(imgGray, ddepth=-1, kernel=Gx)
blured_gauss_Gx = cv2.filter2D(blur_img, ddepth=-1, kernel=Gx)

# Application du filtre Gy
img_Gy = cv2.filter2D(imgGray, ddepth=-1, kernel=Gy)
blured_gauss_Gy = cv2.filter2D(blur_img, ddepth=-1, kernel=Gy)

# Angle du vecteur obtenue par le filtre de Sobel et celui ayant subi un floutage Gaussien et le filtre de Sobel
theta_sobel = np.arctan(img_Gy / (img_Gx + 1e-8)) * (180 / np.pi) + 180 * (img_Gx < 0)
theta_gauss_sobel = np.arctan(blured_gauss_Gy / (blured_gauss_Gx + 1e-10)) * (180 / np.pi) + 180 * (img_Gx < 0)

In [ ]:
print(f"valeur minimale et maximale de theta_sobel = ({theta_sobel.min()};{theta_sobel.max()})")
print(f"valeur minimale et maximale de theta_gausse_sobel = ({theta_gauss_sobel.min()};{theta_gauss_sobel.max()})")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,6))
g0 = axes[0].imshow(theta_sobel, cmap="coolwarm", vmin=0, vmax=359)
g1 = axes[1].imshow(theta_gauss_sobel, cmap="coolwarm", vmin=0, vmax=359)
g2 = axes[2].imshow((theta_sobel - theta_gauss_sobel), cmap="gray", vmin=0, vmax=200)

axes[0].set_title(r"$\theta$ gray convolved with Sobel")
axes[1].set_title(r"$\theta$ gray blured + convolved with Sobel")
axes[2].set_title("Difference")

fig.colorbar(g0, ax=axes[0])
fig.colorbar(g1, ax=axes[1])
fig.colorbar(g2, ax=axes[2])

plt.show()

Nous voyons que le bruit est moindre au sein de l'orientation du vecteur de Sobel quand le flou est appliquée. Néanmoins il y a beaucoup de "résidus"  dans les cases qui sont vides. De plus il semble que les lignes verti

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,6))
g0 = axes[0].imshow(theta_sobel[100:200,100:200], cmap="coolwarm", vmin=0, vmax=270)
g1 = axes[1].imshow(theta_gauss_sobel[100:200,100:200], cmap="coolwarm", vmin=0, vmax=270)
g2 = axes[2].imshow((theta_sobel - theta_gauss_sobel)[100:200,100:200], cmap="gray", vmin=0, vmax=200)

axes[0].set_title(r"$\theta$ gray convolved with Sobel")
axes[1].set_title(r"$\theta$ gray blured + convolved with Sobel")
axes[2].set_title("Difference")

fig.colorbar(g0, ax=axes[0])
fig.colorbar(g1, ax=axes[1])
fig.colorbar(g2, ax=axes[2])

plt.show()

Il semblerait que l'application d'un filtre de floutage Gaussien n'apporte pas une réduction du bruit de l'orientation du vecteur gradient d'intensité que la réduction du bruit au sein de l'amplitude du vecteur.

## Filtre de Canny
[*vidéo intéressante de computerphile*](https://www.youtube.com/watch?v=sRFM5IEqR2w)

L'application du détecteur de Canny se déroule en 4 étapes:
* suppression du bruit
* détermination de l'amplitude et de la direction de la résultante de l'image pour laquelle nous appliquons $G_x$ et $G_y$
* l'application d'une étape de suppression non maximale (ça fait bizarre de traduire)
* Seuillage hystherésis et analyse de connectivité

### Etape 1: Suppression du bruit
Nous avons déjà effectué une suppression du bruit via l'application d'un flou gaussien, profitons en pour essayer une autre technique ***Non Local Mean Denoising***.

In [ ]:
img = Image.open('sudoku_grid.png')
imgGray = np.array(img.convert('L'), dtype='uint8') # nous devons caster en uint8 pour que la méthode fastNlMeanDenoising prenne l'image en output sans erreur
print(f"valeur max/min de imgGray: {imgGray.max()}/{imgGray.min()}")

In [ ]:
blur_img = cv2.fastNlMeansDenoising(imgGray, None, h=5., templateWindowSize=3, searchWindowSize=9)
print(f"valeur max/min de blur_img: {blur_img.max()}/{blur_img.min()}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(imgGray, cmap="gray")
ax[1].imshow(blur_img, cmap="gray")
plt.show()

Cette technique semble très bien marcher !

### Etape 2: Calcul de l'amplitude et de la direction
Pour le filtre de Canny nous avons besoin également des filtres $G_x$ et $G_y$ que nous avons utilisé pour le filtre de Sobel:
$$
G_x = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}
$$

$$
G_y = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
$$

In [ ]:
# Définition des kernel Gx et Gy:
Gx = np.array([[-1, 0, 1],
               [-2, 0, 2],
               [-1, 0, 1]])
Gy = np.array([[-1, -2, -1],
               [ 0,  0,  0],
               [ 1,  2,  1]])

In [ ]:
# Application du filtre Gx
img_Gx = cv2.filter2D(blur_img, ddepth=-1, kernel=Gx)

# Application du filtre Gy
img_Gy = cv2.filter2D(blur_img, ddepth=-1, kernel=Gy)

In [ ]:
print(f"Valeur max/min de img_Gx: {img_Gx.max()}/{img_Gy.min()}")
print(f"Valeur max/min de img_Gy: {img_Gy.max()}/{img_Gy.min()}")

In [ ]:
# Calcul de l'amplitude
amplitude = np.sqrt(img_Gx**2 + img_Gy**2)

# calcul de la direction
theta = np.arctan2(img_Gy, img_Gx) * (180 / np.pi) + 180 * (img_Gx < 0)
print(f"Valeur max de theta: {theta.max()}")
print(f"Valeur min de theta: {theta.min()}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
g1 = ax[0].imshow(amplitude, cmap="gray")
g2 = ax[1].imshow(theta, cmap="coolwarm")

fig.colorbar(g2, ax=ax[1])
plt.show()

### Etape 3: Non maximal suppression gradient
Rappelons le contenue des [slides](https://drive.google.com/file/d/1nEWH_ARdkrsDkLQobdQdrg5GlQtUYZz_/view) du cours de Standford:

> *Edge occurs where gradient reaches a maxima*
> > En réfléchissant sur ces différents points, nous comprenons qu'effectivement un bord peut être représenter par les pixels ayant la valeur de gradient la plus élevée le long de la normale à ce bord (voir figure ci dessous)
> >
> > ![non max suppression 1](non_max_suppression_img1.png)
> >
> > Sur l'image ci-dessus, le bord eut se résumer par les pixels sur lesquels la ligne rouge passe.
>
> *Suppress non-maxima gradient even if it passes threshold*
> > Cela doit être dans le cas où le thresholding hysteresis est effectué en même temps. Mais ça ne sera pas notre cas ici.
>
> *Compare current pixel vs neighbors along direction of gradient*
> > Vous l'avez probablement deviné aussi, nous avons donc besoin de l'amplitude du vecteur ET de la direction. Au sein des slides du CS 131, l'amplitude est plus ou moins appelé **DoG** pour *Difference of Gaussian* ça rappelle peut être quelque chose à certains d'entre vous lorsque vous avez fait l'exercice sur la détection des blobs ;)

![non max suppression 2](non_max_suppression_img2.png)

Pour savoir suivant quelle direction (horizontale, verticale, diagonale montante/descendante) nous utilisons l'image des directions.
Il se peut que la valeur de direction pour le pixel en cours de traitement ne *tombe pas juste*, c'est-à-dire que nous avons des valeurs d'orientation différentes de $0°$, $45°$, $90°$, $125°$, $180°$, $225°$, $270°$, $315°$, $360°$.\
Au moins 2 solutions peuvent s'offrir à nous:
* arrondir à la direction la plus proche
* interpoler les valeurs d'amplitude un piel "avant" et "après" le long de la direction (plus chiant donc on ne le voit pas ici) comme cela est évoqué dans les [slides CS 131](https://drive.google.com/file/d/1nEWH_ARdkrsDkLQobdQdrg5GlQtUYZz_/view)

![non max suppression 2](non_max_suppression_img3.png)

**Side note**: Non Maximal Suppression (noté *NMS*) est aussi une méthode apparaissant lorsque l'on fera de la détection d'objets afin de ne conserver la *bounding box* avec la plus grande confiance. L'idée est semblable, car dans ce cas là nous souhaitons garder que la *bounding box* la plus représentative de la détection et supprimer les autres.


De plus,
> Avoid streaking near threshold value
> * Define two thresholds: Low and High
> * If less than Low, not an edge
> * If greater than High, strong edge
> * If between Low and High, weak edge\
>   *Consider its neighbors iteratively then declare it an “edge pixel” if it is connected to an ‘strong edge pixel’ directly or via pixels between Low and High*

Une fois le *NMS* réalisé, il y aura une étape de seuillage hysthérésis et de connectivité.

In [ ]:
amplitude.dtype

In [ ]:
# fonction NMS (mon implémentation un peu faignante):
dict_kernels = {
    "0": np.array([[0., 0., 0.],[1., 1., 1.],[0., 0., 0.]]),
    "45": np.array([[0., 0., 1.],[0., 1., 0.],[1., 0., 0.]]),
    "90": np.array([[0., 1., 0.],[0., 1., 0.],[0., 1., 0.]]),
    "135": np.array([[1., 0., 0.],[0., 1., 0.],[0., 0., 1.]]),
    "180": np.array([[0., 0., 0.],[1., 1., 1.],[0., 0., 0.]]),
    "225": np.array([[0., 0., 1.],[0., 1., 0.],[1., 0., 0.]]),
    "270": np.array([[0., 1., 0.],[0., 1., 0.],[0., 1., 0.]]),
    "315": np.array([[1., 0., 0.],[0., 1., 0.],[0., 0., 1.]])
}

def nms(amplitude: np.ndarray, angle:np.ndarray):
    line, col = amplitude.shape
    amplitude_nms = np.zeros((line, col), dtype=float)
    rounded_angle = np.zeros((line, col), dtype=int)
    
    rounded_angle[(angle >  337.5) & (angle <= 22.5)] = 0
    rounded_angle[(angle > 22.5) & (angle <= 67.5)] = 45
    rounded_angle[(angle > 67.5) & (angle <= 112.5)] = 90
    rounded_angle[(angle > 112.5) & (angle <= 157.5)] = 135
    rounded_angle[(angle > 157.5) & (angle <= 202.5)] = 180
    rounded_angle[(angle > 202.5) & (angle <= 247.5)] = 225
    rounded_angle[(angle > 247.5) & (angle <= 292.5)] = 270
    rounded_angle[(angle > 292.5) & (angle <= 337.5)] = 315
    # Nous avons un array processé d'angle avec 8 orientations
    
    # Maintenant en fonction de la direction du gradient
    # nous devons regarder le pixel d'intérêt, le pixel avant et le pixel après
    # Attention ça va pas être beau à voir
    for y in range(1,line-1):
        for x in range(1,col-1):
            if rounded_angle[y][x] == 0:
                if ((amplitude[y][x] > amplitude[y][x - 1])  and (amplitude[y][x] > amplitude[y][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 45:
                if ((amplitude[y][x] > amplitude[y - 1][x - 1])  and (amplitude[y][x] > amplitude[y + 1][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 90:
                if ((amplitude[y][x] > amplitude[y - 1][x])  and (amplitude[y][x] > amplitude[y + 1][x])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 135:
                if ((amplitude[y][x] > amplitude[y + 1][x - 1])  and (amplitude[y][x] > amplitude[y - 1][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 180:
                if ((amplitude[y][x] > amplitude[y][x - 1])  and (amplitude[y][x] > amplitude[y][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 225:
                if ((amplitude[y][x] > amplitude[y - 1][x - 1])  and (amplitude[y][x] > amplitude[y + 1][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 270:
                if ((amplitude[y][x] > amplitude[y - 1][x])  and (amplitude[y][x] > amplitude[y + 1][x])):
                    amplitude_nms[y][x] = amplitude[y][x]
            elif rounded_angle[y][x] == 315:
                if ((amplitude[y][x] > amplitude[y + 1][x - 1])  and (amplitude[y][x] > amplitude[y - 1][x + 1])):
                    amplitude_nms[y][x] = amplitude[y][x]
    
    return amplitude_nms

In [ ]:
amplitude_nms = nms(amplitude.astype('float64'), theta)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,20))

g1 = ax[0].imshow(amplitude_nms, cmap="gray")
g2 = ax[1].imshow(amplitude, cmap="gray")

plt.show()

In [ ]:
def hytheresis_tresholding(arr:np.ndarray, high_treshold:float, low_treshold:float):
    hystheresis_arr = np.zeros(arr.shape)
    hystheresis_arr[arr >= high_treshold] = high_treshold
    hystheresis_arr[(arr < high_treshold) & (arr >= high_treshold)] = (high_treshold - low_treshold) / 2
    hystheresis_arr[arr < low_treshold] = 0
    return hystheresis_arr

In [ ]:
def connectivity_tresholding(arr:np.array):
    for i in range(2, arr.shape[0] - 2):
        for j in range(2, arr.shape[1] - 2):
            if (arr[i][j] == 1) and np.any(arr[i - 2 : i + 2][j - 2 : j + 2] == 2):
                arr[i][j] = 2
    return arr

In [ ]:
amplitude_max = amplitude_nms.max()
amplitude_min = amplitude_nms.min()

print(f"valeur max = {amplitude_max}")
print(f"valeur max = {amplitude_min}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,20))

g1 = ax[0].imshow(amplitude_nms, cmap="gray")
g2 = ax[1].imshow(hytheresis_tresholding(amplitude_nms, high_treshold=10, low_treshold=5), cmap="gray")

plt.show()

## Transformée de Hough